<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/telematics_syn_V11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
# Step 1: Load the dataset
dataset = pd.read_csv('/content/drive/My Drive/telematics_syn.csv')

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

# Create the adjusted ClaimYN label
dataset['ClaimYN'] = ((dataset['NB_Claim'] >= 1) & (dataset['AMT_Claim'] > 1000)).astype(int)

# Preprocess the dataset
# 1. Handle missing values
dataset.fillna(method='ffill', inplace=True)  # Simple forward fill for missing values

# 2. Encode categorical variables (if any exist)
categorical_columns = dataset.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    dataset[col] = le.fit_transform(dataset[col])

# 3. Standardize numerical columns
numerical_columns = dataset.drop(columns=['ClaimYN', 'NB_Claim', 'AMT_Claim']).columns
scaler = StandardScaler()
dataset[numerical_columns] = scaler.fit_transform(dataset[numerical_columns])

# Handle the imbalance in the dataset using SMOTE
X = dataset.drop(columns=['ClaimYN'])
y = dataset['ClaimYN']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Merging resampled data back into a single DataFrame
dataset_resampled = pd.concat([X_resampled, y_resampled], axis=1)

# Drop NB_Claim and AMT_Claim columns
dataset_resampled = dataset_resampled.drop(columns=['NB_Claim', 'AMT_Claim'])

# Split the dataset into train, test, and validation sets
from sklearn.model_selection import train_test_split

X = dataset_resampled.drop(columns=['ClaimYN'])
y = dataset_resampled['ClaimYN']

# Split the data (70% train, 15% test, 15% validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")
print(f"Validation set size: {len(X_val)}")

Training set size: 136222
Testing set size: 29191
Validation set size: 29191


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize and train the Logistic Regression model
log_reg = LogisticRegression(random_state=42, max_iter=200)
log_reg.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = log_reg.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75     14593
           1       0.74      0.80      0.77     14598

    accuracy                           0.76     29191
   macro avg       0.76      0.76      0.76     29191
weighted avg       0.76      0.76      0.76     29191



In [7]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = rf.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     14593
           1       1.00      0.98      0.99     14598

    accuracy                           0.99     29191
   macro avg       0.99      0.99      0.99     29191
weighted avg       0.99      0.99      0.99     29191



In [8]:
from xgboost import XGBClassifier

# Initialize and train the XGBoost model
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = xgb.predict(X_test)
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     14593
           1       1.00      0.98      0.99     14598

    accuracy                           0.99     29191
   macro avg       0.99      0.99      0.99     29191
weighted avg       0.99      0.99      0.99     29191



In [9]:
!pip install pytorch_tabnet

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np

# Convert data to numpy arrays for TabNet
X_train_np, y_train_np = X_train.values, y_train.values
X_test_np, y_test_np = X_test.values, y_test.values

# Initialize and train the TabNet model
tabnet = TabNetClassifier(seed=42)
tabnet.fit(X_train_np, y_train_np, eval_set=[(X_test_np, y_test_np)], patience=10)

# Evaluate the model on the test set
y_pred = np.argmax(tabnet.predict_proba(X_test_np), axis=1)
print("TabNet Classification Report:")
print(classification_report(y_test_np, y_pred))

epoch 0  | loss: 0.52504 | val_0_auc: 0.88173 |  0:00:26s
epoch 1  | loss: 0.40063 | val_0_auc: 0.91582 |  0:00:44s
epoch 2  | loss: 0.36553 | val_0_auc: 0.92501 |  0:00:58s
epoch 3  | loss: 0.33912 | val_0_auc: 0.84918 |  0:01:07s
epoch 4  | loss: 0.32147 | val_0_auc: 0.93409 |  0:01:17s
epoch 5  | loss: 0.30906 | val_0_auc: 0.87753 |  0:01:26s
epoch 6  | loss: 0.29658 | val_0_auc: 0.82212 |  0:01:36s
epoch 7  | loss: 0.28286 | val_0_auc: 0.85643 |  0:01:46s
epoch 8  | loss: 0.28663 | val_0_auc: 0.9294  |  0:01:57s
epoch 9  | loss: 0.28628 | val_0_auc: 0.92491 |  0:02:06s
epoch 10 | loss: 0.27638 | val_0_auc: 0.93338 |  0:02:17s
epoch 11 | loss: 0.26571 | val_0_auc: 0.95694 |  0:02:28s
epoch 12 | loss: 0.25292 | val_0_auc: 0.87831 |  0:02:36s
epoch 13 | loss: 0.2587  | val_0_auc: 0.95842 |  0:02:47s
epoch 14 | loss: 0.24295 | val_0_auc: 0.96465 |  0:02:56s
epoch 15 | loss: 0.2294  | val_0_auc: 0.97015 |  0:03:05s
epoch 16 | loss: 0.22512 | val_0_auc: 0.91345 |  0:03:16s
epoch 17 | los

In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.optimizers import Adam

# Reshape data for RNN (3D input: samples, timesteps, features)
X_train_rnn = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_rnn = X_val.values.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_rnn = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Initialize the RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(32, input_shape=(X_train_rnn.shape[1], 1), activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
rnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
rnn_history = rnn_model.fit(X_train_rnn, y_train, epochs=10, batch_size=32,
                            validation_data=(X_val_rnn, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = rnn_model.evaluate(X_test_rnn, y_test)
print(f"RNN Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 53s 12ms/step - accuracy: 0.6979 - loss: 0.5763 - val_accuracy: 0.7704 - val_loss: 0.4868
Epoch 2/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 46s 11ms/step - accuracy: 0.7944 - loss: 0.4473 - val_accuracy: 0.8012 - val_loss: 0.4516
Epoch 3/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 83s 11ms/step - accuracy: 0.8144 - loss: 0.4220 - val_accuracy: 0.8432 - val_loss: 0.3656
Epoch 4/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 47s 11ms/step - accuracy: 0.8323 - loss: 0.3848 - val_accuracy: 0.8571 - val_loss: 0.3268
Epoch 5/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 47s 11ms/step - accuracy: 0.8512 - loss: 0.3437 - val_accuracy: 0.8757 - val_loss: 0.2834
Epoch 6/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 47s 11ms/step - accuracy: 0.7880 - loss: 0.4017 - val_accuracy: 0.7543 - val_loss: 0.5500
Epoch 7/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 47s 11ms/step - accuracy: 0.8249 - loss: 0.3867 - val_accuracy: 0.8662 - val_loss: 0.2930
Epoch 8/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 81s 11ms/step - accuracy: 0.8624 -

In [14]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

# Initialize the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_rnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
cnn_history = cnn_model.fit(X_train_rnn, y_train, epochs=10, batch_size=32,
                            validation_data=(X_val_rnn, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(X_test_rnn, y_test)
print(f"CNN Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8130 - loss: 0.4153 - val_accuracy: 0.8544 - val_loss: 0.3278
Epoch 2/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8602 - loss: 0.3178 - val_accuracy: 0.8651 - val_loss: 0.3048
Epoch 3/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8696 - loss: 0.2975 - val_accuracy: 0.8744 - val_loss: 0.2873
Epoch 4/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8768 - loss: 0.2827 - val_accuracy: 0.8785 - val_loss: 0.2738
Epoch 5/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8804 - loss: 0.2727 - val_accuracy: 0.8816 - val_loss: 0.2712
Epoch 6/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8849 - loss: 0.2638 - val_accuracy: 0.8876 - val_loss: 0.2649
Epoch 7/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8884 - loss: 0.2563 - val_accuracy: 0.8899 - val_loss: 0.2534
Epoch 8/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.8917 - loss: 0

In [15]:
from keras.layers import LSTM

# Initialize the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(32, input_shape=(X_train_rnn.shape[1], 1), activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile the model
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_history = lstm_model.fit(X_train_rnn, y_train, epochs=10, batch_size=32,
                              validation_data=(X_val_rnn, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = lstm_model.evaluate(X_test_rnn, y_test)
print(f"LSTM Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 97s 22ms/step - accuracy: 0.6749 - loss: 0.7676 - val_accuracy: 0.7551 - val_loss: 0.5074
Epoch 2/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 91s 21ms/step - accuracy: 0.7585 - loss: 0.5019 - val_accuracy: 0.7838 - val_loss: 0.4617
Epoch 3/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 148s 23ms/step - accuracy: 0.7644 - loss: 0.4958 - val_accuracy: 0.7904 - val_loss: 0.4571
Epoch 4/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 91s 21ms/step - accuracy: 0.7938 - loss: 0.4468 - val_accuracy: 0.8223 - val_loss: 0.3889
Epoch 5/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 148s 23ms/step - accuracy: 0.7871 - loss: 0.4680 - val_accuracy: 0.8207 - val_loss: 0.3889
Epoch 6/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 137s 22ms/step - accuracy: 0.8198 - loss: 0.3887 - val_accuracy: 0.8312 - val_loss: 0.3665
Epoch 7/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 92s 22ms/step - accuracy: 0.8237 - loss: 0.3798 - val_accuracy: 0.8453 - val_loss: 0.3371
Epoch 8/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 93s 22ms/step - accuracy: 0.843

In [16]:
from keras.layers import GRU

# Initialize the GRU model
gru_model = Sequential()
gru_model.add(GRU(32, input_shape=(X_train_rnn.shape[1], 1), activation='relu'))
gru_model.add(Dense(1, activation='sigmoid'))

# Compile the model
gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
gru_history = gru_model.fit(X_train_rnn, y_train, epochs=10, batch_size=32,
                            validation_data=(X_val_rnn, y_val), verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = gru_model.evaluate(X_test_rnn, y_test)
print(f"GRU Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 122s 28ms/step - accuracy: 0.7182 - loss: 0.5350 - val_accuracy: 0.8042 - val_loss: 0.4264
Epoch 2/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 143s 28ms/step - accuracy: 0.8101 - loss: 0.4099 - val_accuracy: 0.8147 - val_loss: 0.3937
Epoch 3/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 139s 27ms/step - accuracy: 0.8390 - loss: 0.3489 - val_accuracy: 0.8725 - val_loss: 0.2816
Epoch 4/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 144s 28ms/step - accuracy: 0.8679 - loss: 0.2862 - val_accuracy: 0.8710 - val_loss: 0.2758
Epoch 5/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 139s 27ms/step - accuracy: 0.8840 - loss: 0.2579 - val_accuracy: 0.8850 - val_loss: 0.2583
Epoch 6/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 146s 28ms/step - accuracy: 0.8902 - loss: 0.2436 - val_accuracy: 0.8994 - val_loss: 0.2255
Epoch 7/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 116s 27ms/step - accuracy: 0.8960 - loss: 0.2320 - val_accuracy: 0.9020 - val_loss: 0.2204
Epoch 8/10
4257/4257 ━━━━━━━━━━━━━━━━━━━━ 144s 28ms/step - accuracy: 

## Import Necessary Libraries for Evaluation

In [17]:
from sklearn.metrics import matthews_corrcoef, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure you have seaborn for nicer plots
sns.set(style="whitegrid")

In [18]:
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """Evaluate the model using MCC, AUC, and plot the confusion matrix."""

    # Calculate MCC
    mcc = matthews_corrcoef(y_true, y_pred)

    # Calculate AUC
    auc = roc_auc_score(y_true, y_pred_proba)

    # Print results
    print(f"{model_name} Evaluation:")
    print(f"  MCC: {mcc:.4f}")
    print(f"  AUC: {auc:.4f}")

    # Plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

def plot_training_history(history, model_name):
    """Plot training & validation accuracy and loss from the model's history."""
    # Accuracy
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()